In [1]:
# imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import metrics
from env import db,host,password,protocol,user
import os
import prepare


# Initial Questions
I have some questions for you that I need to be answered before the board meeting Thursday afternoon. I need to be able to speak to the following questions. I also need a single slide that I can incorporate into my existing presentation (Google Slides) that summarizes the most important points. My questions are listed below; however, if you discover anything else important that I didn’t think to ask, please include that as well.


1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?
2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?
3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?
4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?
5. At some point in 2019, the ability for students and alumni to access both curriculums (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before?
6. What topics are grads continuing to reference after graduation and into their jobs (for each program)?
7. Which lessons are least accessed?
8. Anything else I should be aware of?

In [2]:
# acquire log data
df1 = prepare.get_logs()

In [3]:
# quick look at log data
df1.head()

,Unnamed: 0,date,time,path,user_id,cohort_id,ip
0,0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61
1,1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61
2,2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61
3,3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61
4,4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61


In [4]:
# created column to hold Data Science or Web Dev cohort
df1['cohorts']=df1['cohort_id']

# updated cohort_id values to match Data Science(DS) searches
df1['cohorts'] = df1['cohorts'].replace({34.0:'DS',28.0:'DS',137.0:'DS',25.0:'DS',59.0:'DS',55.0:'DS',24.0:'DS',133.0:'DS'})

In [5]:
# updated cohort_id values to match Web Dev(WD) searches
df1['cohorts'] = df1['cohorts'].replace({5.0:'WD',4.0:'WD',9.0:'WD',6.0:'WD',2.0:'WD',11.0:'WD',12.0:'WD',7.0:'WD',15.0:'WD',19.0:'WD',18.0:'WD',13.0:'WD',17.0:'WD',1.0:'WD',14.0:'WD',61.0:'WD',27.0:'WD',31.0:'WD',26.0:'WD',52.0:'WD',32.0:'WD',56.0:'WD',53.0:'WD',29.0:'WD',33.0:'WD',58.0:'WD',16.0:'WD',23.0:'WD',57.0:'WD',139.0:'WD',51.0:'WD',132.0:'WD',138:'WD', 21.0:'WD',8.0:'WD',28.0:'WD',22.0:'WD',134.0:'WD', 135.0:'WD',62:'WD',135.0:'WD',133.0:'WD'})

In [6]:
# value counts for each cohort_id
df1.cohort_id.value_counts().head()

28.0    84031
33.0    40730
29.0    38096
62.0    37109
53.0    36902
Name: cohort_id, dtype: int64

In [7]:
# data science and web dev count breakdown 
df1.cohorts.value_counts()

WD    598665
DS    248665
Name: cohorts, dtype: int64

In [8]:
# calculated path visits
visit_counts = df1['path'].value_counts()

# created a column for visit per path
df1['visits'] = df1['path'].map(visit_counts)

In [9]:
# grouped data by path and cohort, and added the total visits
grouped = df1.groupby(['path', 'cohort_id'])['visits'].sum().reset_index()

# group the data by path, and calculate the mean visits across all cohorts
grouped_mean = grouped.groupby('path')['visits'].mean().reset_index()

# sort the data by mean visits in descending order
sorted_data = grouped_mean.sort_values(by='visits', ascending=False)


# 1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?
##  javascript-i

In [10]:
sorted_data

,path,visits
6,/,4.908622e+07
1712,javascript-i,8.638689e+06
2007,search/search_index.json,8.347467e+06
2190,toc,8.046563e+06
1685,java-iii,4.758123e+06
...,...,...
1506,extra-,1.000000e+00
1507,extra-challenges,1.000000e+00
1513,extra-exercises/j,1.000000e+00
1514,extra-features,1.000000e+00


# 2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?
## DS Cohort 28

In [11]:
# group the data by cohort and lesson_referral, and calculate the counts
grouped = df1.groupby(['cohort_id', 'path']).size().reset_index(name='visits')

# pivot the data to create a contingency table
cont_table = pd.pivot_table(grouped, values='visits', index='cohort_id', columns='path', fill_value=0)


In [12]:
# table breakdown by cohort and path
cont_table.head()

path,%20https://github.com/RaulCPena,',",%20https://github.com/RaulCPena",.git,.gitignore,.well-known/assetlinks.json,/,00_,00_index,01_intro,...,web-design/ui/visuals,web-design/ux,web-design/ux/layout,web-design/ux/layout/.json,web-design/ux/purpose,web-dev-day-two,where,working-with-time-series-data,wp-admin,wp-login
cohort_id,,,,,,,,,,,,,,,,,,,,,
1.0,0,0,0,0,0,0,626,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2.0,0,0,0,0,0,0,17,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4.0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5.0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6.0,0,0,0,0,0,0,4,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
# created variable to hold the cohort with the highest number of lesson searches
max_cohort = cont_table.sum(axis=1).idxmax()
# view max_cohort
max_cohort

28.0

# 3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?
## user_id 685, 354, and 64 have only accessed sites once during class hours

In [14]:
# group the data by user_id and sum the access counts
user_visit_counts = df1.groupby(['user_id','time'])['visits'].sum()

# sort the user access counts in descending order
visit_counts = user_visit_counts.sort_values(ascending=False)
visit_counts

user_id  time    
580      11:58:53    402504.0
         21:42:46    352191.0
         12:25:51    251565.0
         07:32:55    251565.0
         09:03:06    201591.0
                       ...   
617      19:04:23         1.0
685      16:33:49         1.0
354      15:50:30         1.0
64       11:09:11         1.0
         16:36:11         1.0
Name: visits, Length: 849706, dtype: float64

# 4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?

## Possible suspicious activity low visit ips at times outside of usual class. 

In [16]:
# verify ip addresses by visit counts
ip_activity = df1.groupby('ip')['visits'].nunique()

# set suspicious ip activity for anything less than 10 visits
suspicious_ip = ip_activity[ip_activity < 10]

# view ip addresses
suspicious_ip

ip
100.35.67.162      8
103.137.12.164     8
103.208.220.130    1
103.208.220.131    1
103.208.220.132    1
                  ..
99.6.251.67        7
99.61.71.158       1
99.66.104.96       3
99.76.233.227      1
99.98.245.245      2
Name: visits, Length: 3559, dtype: int64

In [39]:
# verify ip addresses and time by visit counts
ip_activity = df1.groupby(['ip','time'])['visits'].nunique()

# set suspicious ip activity for anything less than 10 visits
suspicious_ip = ip_activity[ip_activity < 10]

# view ip addresses
suspicious_ip

ip              time    
100.11.170.135  06:06:47    1
                06:07:18    1
                08:55:02    1
                08:55:13    1
                09:03:44    1
                           ..
99.98.36.168    23:49:37    1
                23:49:40    1
                23:49:45    1
                23:49:57    1
                23:50:03    1
Name: visits, Length: 578001, dtype: int64

# 7. Which lessons are least accessed?
## extra-challenges, extra-exercises, extra-features


In [42]:
# grouped data by path and cohort, and added the total visits
grouped = df1.groupby(['path', 'cohort_id'])['visits'].sum().reset_index()

# group the data by path, and calculate the mean visits across all cohorts
grouped_mean = grouped.groupby('path')['visits'].mean().reset_index()

# sort the data by mean visits in descending order
sorted_data = grouped_mean.sort_values(by='visits', ascending=False)

#view path by visits
sorted_data

,path,visits
6,/,4.908622e+07
1712,javascript-i,8.638689e+06
2007,search/search_index.json,8.347467e+06
2190,toc,8.046563e+06
1685,java-iii,4.758123e+06
...,...,...
1506,extra-,1.000000e+00
1507,extra-challenges,1.000000e+00
1513,extra-exercises/j,1.000000e+00
1514,extra-features,1.000000e+00


# Cohorts DF

In [17]:
# aquire cohorts data
df = prepare.get_cohorts()

In [18]:
# look at data 
df.head()

,Unnamed: 0,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
0,0,1,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
1,1,2,Badlands,#badlands,2014-06-04,2014-08-22,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
2,2,3,Carlsbad,#carlsbad,2014-09-04,2014-11-05,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
3,3,4,Denali,#denali,2014-10-20,2015-01-18,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
4,4,5,Everglades,#everglades,2014-11-18,2015-02-24,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1


In [21]:
#verify program breakdown
df.program_id.value_counts()

2    31
1    14
3     6
4     2
Name: program_id, dtype: int64

In [22]:
# sorting names to find distinction in program id
df[df['name']=='Staff']

,Unnamed: 0,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
27,27,28,Staff,#,2014-02-04,2014-02-04,2018-12-06 17:04:19,2018-12-06 17:04:19,NaN,2
